In [2]:
# open sample data from sample_data.txt
with open('sample_data.txt', 'r') as file:
    sample_data = file.read()

In [4]:
def parse_relationships(text):
    """Parse the relationship text into structured data"""
    relationships = []
    sections = text.strip().split('\n\n')

    for section in sections:
        lines = section.strip().split('\n')
        if len(lines) >= 3:
            from_concept = lines[0].replace('FROM: ', '').strip()
            to_concept = lines[1].replace('TO: ', '').strip()
            relationship = lines[2].replace('RELATIONSHIP: ', '').strip()

            relationships.append({
                'from': from_concept,
                'to': to_concept,
                'relationship': relationship,
                'weight': len(relationship.split())  # Use relationship length as weight
            })

    return relationships


relationships = parse_relationships(sample_data)

relationships

[{'from': 'Graphene nanoplatelets (GNPs)',
  'to': 'Neuronal differentiation',
  'relationship': 'GNPs enhance the expression of neuronal markers MAP2, Nestin, and Tuj1, promoting the differentiation of human bone marrow mesenchymal stem cells (hBMSCs) into neurons.',
  'weight': 24},
 {'from': 'GNP concentration',
  'to': 'Cell viability',
  'relationship': 'Lower concentrations of GNPs maintained cell viability, whereas higher concentrations were detrimental to hBMSCs.',
  'weight': 14},
 {'from': '0.4 µg/ml GNP coating',
  'to': 'Morphological changes',
  'relationship': 'This specific concentration leads to observable morphological changes and increased fluorescence in hBMSC cultures, indicating enhanced differentiation.',
  'weight': 17},
 {'from': 'Calcium imaging with Fluo4-AM',
  'to': 'Neuronal activity assessment',
  'relationship': "This method is used to show increased neuronal activity, underscoring GNPs' role in neuronal maturation.",
  'weight': 15},
 {'from': 'GNPs',
  

In [14]:
import networkx as nx
import plotly.graph_objects as go

layout_type='spring'

# def create_network_graph(relationships, layout_type="spring"):
"""Create a network graph using plotly"""
G = nx.DiGraph()

# Add edges with weights
for rel in relationships:
    G.add_edge(rel['from'], rel['to'],
                weight=rel['weight'],
                relationship=rel['relationship'])

# Calculate layout
if layout_type == "spring":
    pos = nx.spring_layout(G, k=3, iterations=50)
elif layout_type == "circular":
    pos = nx.circular_layout(G)
elif layout_type == "hierarchical":
    pos = nx.shell_layout(G)
else:
    pos = nx.random_layout(G)

# Calculate node sizes based on degree
node_degrees = dict(G.degree())
max_degree = max(node_degrees.values()) if node_degrees else 1

# Create edge traces
edge_x = []
edge_y = []
edge_info = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_info.append(G[edge[0]][edge[1]]['relationship'])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color='rgba(125,125,125,0.5)'),
    hoverinfo='none',
    mode='lines'
)

# Create node traces
node_x = []
node_y = []
node_text = []
node_info = []
node_sizes = []
node_colors = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)

    # Node info for hover
    adjacencies = list(G.neighbors(node))
    node_info.append(f'{node}<br>Connections: {len(adjacencies)}<br>' +
                        f'Connected to: {", ".join(adjacencies[:3])}{"..." if len(adjacencies) > 3 else ""}')

    # Node size based on degree
    degree = node_degrees[node]
    node_sizes.append(20 + (degree / max_degree) * 30)

    # Node color based on type (simple heuristic)
    if any(word in node.lower() for word in ['gnp', 'graphene']):
        node_colors.append('#ff7f0e')  # Orange for GNP-related
    elif any(word in node.lower() for word in ['neuron', 'neural', 'differentiation']):
        node_colors.append('#2ca02c')  # Green for neural-related
    elif any(word in node.lower() for word in ['assay', 'imaging', 'method']):
        node_colors.append('#d62728')  # Red for methods
    else:
        node_colors.append('#1f77b4')  # Blue for others

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text=node_text,
    hovertext=node_info,
    textposition="middle center",
    textfont=dict(size=10, color="white"),
    marker=dict(
        size=node_sizes,
        color=node_colors,
        line=dict(width=2, color="white"),
        opacity=0.8
    )
)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title=dict(
                        text='Research Paper Concept Network',
                        x=0.5,
                        font=dict(size=20)
                    ),
                    # titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="Node size indicates connection degree. Colors represent concept categories.",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002,
                        xanchor="left", yanchor="bottom",
                        font=dict(color="gray", size=12)
                    )],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    plot_bgcolor='white'
                ))

# return fig

In [17]:
fig.write_html("network_graph.html")
